# Planning observations with `astroplan`
# by Toby Smith
http://tobyrsmith.github.io/

In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord

from astroplan import Observer, FixedTarget

In [2]:
import astroplan
astroplan.test()

============================= test session starts ==============================
platform darwin -- Python 3.6.8, pytest-4.3.1, py-1.8.0, pluggy-0.9.0

Running tests with astroplan version 0.5.
Running tests in //anaconda/envs/py34/lib/python3.6/site-packages/astroplan.

Date: 2020-02-13T17:50:48

Platform: Darwin-17.7.0-x86_64-i386-64bit

Executable: //anaconda/envs/py34/bin/python

Full Python Version: 
3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]

encodings: sys: utf-8, locale: UTF-8, filesystem: utf-8
byteorder: little
float info: dig: 15, mant_dig: 15

Numpy: 1.16.2
Scipy: 1.2.1
Matplotlib: 3.0.3
Pandas: 0.24.2
Astropy: 3.2.3
pytz: 2018.9
pyephem: 3.7.7.0
matplotlib: 3.0.3
pytest-mpl: not available
astropy_helpers: 3.2.2
Using Astropy options: remote_data: none.

rootdir: /Users/lockepatton/Desktop/2020-ASTR100/Notes/2020_02_13_astropy_units_and_time, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.2, doctes

0

## Time and Dates
- ### All dates and times in are UTC: *Coordinated Universal Time* 
- All `Time` calculation assume that the time is UTC
- UTC is related to  Greenwich Mean Time (GMT) but does not change with a change of seasons.

In [3]:
date1 = Time("2020-02-13 00:00:00", format='iso')

print(date1)

2020-02-13 00:00:00.000


### Current UTC Time

In [4]:
now = Time.now()    # Current UTC Time

print(now)

2020-02-13 22:51:29.224722


### Different Date Formats

In [5]:
print(now.jd)               # Julian Date

print(now.mjd)              # Modified Julian Date

print(now.unix)             # Seconds since the unix epoch (Jan 01, 1970 00:00:00 UTC)

print(now.decimalyear)      # Fraction of the year (very useful for plotting)

2458893.4524215823
58892.95242158243
1581634289.2247221
2020.1200885835585


### Math with Time and Dates

The most common unints in working with dates and times are: u.s (seconds), u.min (minutes), u.h (hours), u.d (days), and u.yr (years)

### Converting to local time - Need to know your offset from UTC

In [6]:
# Seattle Time (PST) = UTC - 8 hours

offset = -8 * u.h

print("The current local time is {0} local time".format(now + offset))

The current local time is 2020-02-13 14:51:29.224722 local time


In [7]:
print("In 1 hour and 25 minutes it will be {0} UTC".format(now + 1*u.h + 25*u.min))

In 1 hour and 25 minutes it will be 2020-02-14 00:16:29.224722 UTC


In [8]:
Observing_Week = Time("2020-03-16 00:00:00", format='iso')

dt = Observing_Week - now

print(dt.to(u.d))   # difference in days

print(dt.to(u.s))  #difference in seconds

31.047578417569444 d
2682510.7752779997 s


---

### [Accurate Time](http://bmmorris.blogspot.com/2015/06/ut1-utc-and-astropy.html) - `UT1`

`AstroPy` calculates the times of events to a very high accuracy. To do this, is has to account for the fact that  Earth's rotation period is constantly changing due to tidal forces and changes in the Earth's moment of inertia.

To do this, `AstroPy` uses a time convention called `UT1`. This system is tied to the rotation of the Earth with repect to the positions of distant quasars. Since the Earth's rotation is constantly changing, the time system `UT1` is constanly changing with repect to `UTC`. 

The orientation of the Earth, which must be measured continuously to keep `UT1` accurate. This measurement is logged by the International Earth Rotation and Reference Systems Service (IERS). They update a "bulletin" with the most recent measurements of the Earth's orientation. This bulletin is constantly being updated.

You will run into occasions when you will get a warning that your dates are out of range of the IERS bulletin. To update the bulletin, run the follow block of code:

---

In [9]:
from astroplan import download_IERS_A

download_IERS_A()

HTTPError: HTTP Error 403: Forbidden

## Setting your location - `Observer`

In [ ]:
astrolab = Observer(longitude = -122.3115 * u.deg,
                    latitude = 47.65371 * u.deg,
                    elevation = 63.4 * u.m,
                    name = "Astrolab"
                    )

# astrolab = Observer(longitude = 42.361145 * u.deg,
#                     latitude = -71.057083 * u.deg,
#                     elevation = 6 * u.m,
#                     name = "Astrolab"
#                     )

### Information at your location

In [ ]:
sunset_here = astrolab.sun_set_time(now, which='nearest')
sunrise_here = astrolab.sun_rise_time(now, which='next')
midnight_here = astrolab.midnight(now, which='next')

print("Sunset will be at {0.iso} UTC".format(sunset_here))
print("Local Midnight will be at {0.iso} UTC".format(midnight_here))
print("Sunrise will be at {0.iso} UTC".format(sunrise_here))

In [ ]:
print("Sunset will be at {0.iso} local time".format(sunset_here + offset))
print("Local Midnight will be at {0.iso} local time".format(midnight_here + offset))
print("Sunrise will be at {0.iso} local time".format(sunrise_here + offset))

#### The Manastash Ridge Observatory (MRO) is operated by the Astronomy Department of the University of Washington for the training of graduate and undergraduate students as well as for astronomical research.

In [ ]:
mro = Observer.at_site('mro')

sunset_mro = mro.sun_set_time(now, which='nearest')

print("Sunset at MRO will be at {0.iso} local time".format(sunset_mro + offset))

In [ ]:
(sunset_here - sunset_mro).to(u.min)

In [ ]:
midnight_mro = mro.midnight(now, which='next')

astrolab.local_sidereal_time(midnight_mro)

In [ ]:
# Astronomical twilight is when the Sun is 18 degrees below the horizon

astro_set = mro.twilight_evening_astronomical(now, which='nearest')  
astro_rise = mro.twilight_morning_astronomical(now, which='next')

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set))
print("Astronomical Midnight is at {0.iso} UTC".format(midnight_mro))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise))

In [ ]:
# Local Times

print("Astronomical Evening Twilight starts at {0.iso} local time".format(astro_set + offset))
print("Astronomical Midnight is at {0.iso} local time".format(midnight_mro - 8*u.h))
print("Astronomical Morning Twilight starts at {0.iso} local time".format(astro_rise + offset))

## Objects in the sky - `FixedTarget`

### You can define targets by [coordinates](./Astro_Coordinates.pdf)

In [ ]:
coords = SkyCoord('02h19m00.0s', '+57d07m042s', frame='icrs')
ngc869 = FixedTarget(name='NGC869', coord=coords)

In [ ]:
ngc869.ra

In [ ]:
ngc869.ra.hms

In [ ]:
astrolab.target_is_up(midnight_here, ngc869)

In [ ]:
# Altitude and Azimuth of a target at a specific time

aa = astrolab.altaz(midnight_here, ngc869)

aa.alt.degree, aa.az.degree

In [ ]:
# You can get the galactice coords of the target

aa.galactic

In [ ]:
# You can get the coords at a different epoch (1950)

aa.fk4

### Most targets can be defined by name

In [ ]:
Obj1 = FixedTarget.from_name("m1")

In [ ]:
Obj1.coord 

In [ ]:
Obj1.ra.hms